In [1]:
import os
os.chdir('..')
print(os.getcwd())

/Users/roise0r/intellij-projects/yc-chatbot


#### These are definitions responsible for pulling data from HackerNews:

In [2]:
import json
import os
import time
from datetime import datetime, timedelta

import pytz
import requests
from bs4 import BeautifulSoup
from requests.exceptions import HTTPError, SSLError

# tz_str = "UTC"
tz_str = "US/Pacific"
# tz_str = "America/Los_Angeles"
os.environ['TZ'] = tz_str
time.tzset()
tz_ = pytz.timezone(tz_str)


def extract_text_from_url(url):
    response = requests.get(url, timeout=10)
    response.raise_for_status()  # Raise an error for bad status codes
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract text from the main content of the page.
    # This may vary depending on the structure of the webpage.
    paragraphs = soup.find_all('p')
    page_text = ' '.join([p.get_text() for p in paragraphs])
    return page_text


def fetch_hacker_news_stories(
    store_every,
    stories_cutoff_in_days,
    min_chars_per_story,
):
    start_dt = datetime.now()
    base_url = "https://hacker-news.firebaseio.com/v0"
    max_item_id = requests.get(f"{base_url}/maxitem.json", timeout=10).json()

    # Get the current time and calculate the cutoff time for the 'days' parameter.
    current_time = datetime.now()
    cutoff_time = current_time - timedelta(days=stories_cutoff_in_days)
    # The cutoff day should be a full day, and included.
    cutoff_time = cutoff_time.replace(hour=0, minute=0, second=0, microsecond=0)
    cutoff_timestamp = int(cutoff_time.timestamp())

    stories = []
    for item_id in range(max_item_id, 0, -store_every):

        item_url = f"{base_url}/item/{item_id}.json"
        try:
            item_data = requests.get(item_url, timeout=10).json()
        except requests.exceptions.Timeout:
            print(f"WARNING: Timeout for item_data for item_id: {item_id}")
            continue
        except requests.exceptions.ConnectionError:
            print(f"WARNING: ConnectionError for item_data for item_id: {item_id}")
            continue

        # Check if the item's time is less then the cutoff. If that's the case,
        #   we can break early, since we know that any stories fetched from now
        #   on will have a 'time' less than the cutoff as well, since we started
        #   with the item with the largest item_id and going backwards.
        if item_data['time'] < cutoff_timestamp:
            print("Reached cutoff time.")
            break

        # Skip this item if it's not a story or if it doesn't have an URL.
        if item_data['type'] != 'story' or 'url' not in item_data:
            continue

        # Try fetching and extract text from the URL. If HTTPError is raised, skip.
        try:
            item_data['text'] = (
                f"Story Title: {item_data['title']} -- Story text: {extract_text_from_url(item_data['url'])}"
            )
        except HTTPError:
            print(f"WARNING: HTTPError for story_id: {item_id}")
            continue
        except SSLError:
            print(f"WARNING: SSLError for story_id: {item_id}")
            continue
        except requests.exceptions.Timeout:
            print(f"WARNING: Timeout for extract_text for story_id: {item_id}")
            continue
        except requests.exceptions.ConnectionError:
            print(f"WARNING: ConnectionError for extract_text for item_id: {item_id}")
            continue

        # Include only stories that have at least some characters.
        if len(item_data['text']) < min_chars_per_story:
            print(f"WARNING: Min char reqs not met for story_id: {item_id}")
            continue

        # Add datetime field based on timestamp.
        item_data["dt"] = datetime.fromtimestamp(item_data["time"], tz=tz_).isoformat()

        stories.append(item_data)

        # Checkpoint.
        if len(stories) % 10 == 0:
            print(f"Stories fetched so far: {len(stories)}")

    print(f"Total stories fetched: {len(stories)}")
    print(f"Runtime: {datetime.now() - start_dt}")

    return stories

#### The main method for the definitions above:

In [3]:
## args being (we could use `argparse`, but no point).

# In order to be able to finish fetching stories for all days in a reasonable
#   amount of time, we need to only fetch every other story or so.
store_every = 1
# Get stories for this amount of days in the past, including today. Last day
#   will have fewer stories than other days.
stories_cutoff_in_days = 3
# Don't save stories that have fewer than 100 chars.
min_chars_per_story = 100

## args end

hacker_news_stories = fetch_hacker_news_stories(
    store_every=store_every,
    stories_cutoff_in_days=stories_cutoff_in_days,
    min_chars_per_story=min_chars_per_story,
)

file_path = 'hacker_news_stories.json'
with open(file_path, 'w') as json_file:
    # noinspection PyTypeChecker
    json.dump(hacker_news_stories, json_file)
print(f"Stories written to {file_path}")

Reached cutoff time.
Total stories fetched: 2
Runtime: 0:00:14.654571
Stories written to hacker_news_stories.json


#### Next is store_embeddings.py -- It's responsible for getting all the info from the JSON we created and ingest all that into chromadb

In [4]:
import json
import os
import warnings
from datetime import datetime

import chromadb
import nltk
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer

from modules.memory import Memory

warnings.simplefilter(action='ignore', category=FutureWarning)

# Load stop words. this needs to be outside functions so it doesn't download each time. duh!
nltk.download('stopwords', download_dir=os.getcwd())
nltk.data.path = [os.getcwd()]
stop_words = set(stopwords.words('english'))


def clean_text(text):

    cleaned_text = text.lower()

    cleaned_text = cleaned_text.replace("story title: ", " ")
    cleaned_text = cleaned_text.replace("story text: ", " ")
    cleaned_text = cleaned_text.replace("--", " ")

    cleaned_text = ' '.join(_ for _ in cleaned_text.split() if _ not in stop_words)
    return cleaned_text


def generate_embeddings(data):

    model = SentenceTransformer('all-MiniLM-L6-v2')

    cleaned_texts = [clean_text(story['text']) for story in data]
    embeddings = model.encode(cleaned_texts, show_progress_bar=True)
    return cleaned_texts, embeddings


def store_embeddings(hacker_news_stories, use_cleaned_text, cleaned_texts, embeddings):

    chromadb_client = chromadb.PersistentClient(path="chromadb")

    # Check if the collection already exists and drop it
    if "hacker_news" in [collection.name for collection in chromadb_client.list_collections()]:
        chromadb_client.delete_collection("hacker_news")
    collection = chromadb_client.create_collection("hacker_news")

    for i, (story, cleaned_text, embedding) in enumerate(zip(hacker_news_stories, cleaned_texts, embeddings)):
        collection.add(
            metadatas=[{
                "date": story["dt"][:10],
                "timestamp": story["time"],
            }],
            documents=[cleaned_text if use_cleaned_text else story['text']],
            embeddings=[embedding.tolist()],
            ids=[f"id_{i}"],
        )

    # Check some embeddings.
    embeddings_ls = [embedding.tolist() for embedding in embeddings]
    # n_results=10 is only for sanity-checking the saved stories, so it doesn't matter.
    results = collection.query(query_embeddings=[embeddings_ls[0]], n_results=10)
    print(json.dumps(results))

/Users/roise0r/Library/Caches/pypoetry/virtualenvs/yc-chatbot-Yp3Islnz-py3.10/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package stopwords to /Users/roise0r/intellij-
[nltk_data]     projects/yc-chatbot...
[nltk_data]   Package stopwords is already up-to-date!


#### and its main method:

In [ ]:
## args being

# Whether to use the cleaned text (remove stop words, lowecase, etc.), or use original text to store in chromadb.
use_cleaned_text = False

## args end

memory = Memory()
memory.log_memory(print, "before")
dt_start = datetime.now()

file_path = 'hacker_news_stories.json'
with open(file_path, 'r') as file:
    hacker_news_stories = json.load(file)
    print(f"Total stories to be stored: {len(hacker_news_stories)}")

cleaned_texts, embeddings = generate_embeddings(hacker_news_stories)
memory.log_memory(print, "after_generate")

print(f"type embeddings: {type(embeddings)}")
print(f"len embeddings: {len(embeddings)}")
print(f"size embeddings: {embeddings.size}")
print(f"embeddings: {embeddings}")
print(f"len embeddings 0: {len(embeddings[0])}")
print(f"len embeddings 1: {len(embeddings[1])}")
print(f"len embeddings 2: {len(embeddings[2])}")

store_embeddings(
    hacker_news_stories,
    use_cleaned_text,
    cleaned_texts,
    embeddings,
)
memory.log_memory(print, "after_store")

print(f"Total runtime: {datetime.now() - dt_start}")